In [1]:
import requests
import csv
from bs4 import BeautifulSoup

url = 'http://www.taiwanrate.com/'

headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'
}

response = requests.get(url, headers=headers)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'lxml')

table = soup.select('table#table1')[0]

with open('TaiwanRate.csv', mode='w', newline='') as csv_file:
    
    writer = csv.writer(csv_file)
    
    fieldnames = [td.text.strip() for td in table.select('tr.bodytablehead td')]
    writer.writerow(fieldnames)
    
    for tr in table.select('tr')[1:]:
        writer.writerow([td.text for td in tr.select('td')])

SQLite 下載連結：https://www.sqlite.org/download.html
DB Browser for SQLite 下載連結：https://sqlitebrowser.org/dl/

## 股票清單爬蟲

本國上市證券國際證券辨識號碼一覽表：<br>https://isin.twse.com.tw/isin/C_public.jsp?strMode=2<br><br>
本國上櫃證券國際證券辨識號碼一覽表：<br>https://isin.twse.com.tw/isin/C_public.jsp?strMode=4<br><br>
本國興櫃證券國際證券辨識號碼一覽表：<br>https://isin.twse.com.tw/isin/C_public.jsp?strMode=5

In [2]:
import requests
import sqlite3
from bs4 import BeautifulSoup
from user_agent import generate_user_agent

connection = sqlite3.connect('上市股票清單.db')
cursor = connection.cursor()

command = '''
CREATE TABLE IF NOT EXISTS
stock(id TEXT PRIMARY KEY, name TEXT, date TEXT)
'''

cursor.execute(command)
cursor.execute('DELETE FROM stock')


url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=2'

headers = {'user-agent': generate_user_agent()}

response = requests.get(url, headers=headers)
# response.text[:2000] # 顯示全部內容可能造成記憶體不足，所以只顯示前2000個字元

soup = BeautifulSoup(response.text, 'lxml')

table = soup.select('table')[1]
# table # 顯示表格內容可能造成記憶體不足

for tr in table.select('tr')[2:]:
    
    try:
        stock_id = tr.select('td')[0].text.split('\u3000')[0]
        stock_name = tr.select('td')[0].text.split('\u3000')[1]
        stock_date = tr.select('td')[2].text

        cursor.execute('INSERT INTO stock VALUES ("{}", "{}", "{}")'.format(stock_id, stock_name, stock_date))
    except:
        print(tr)
        
connection.commit()

KeyboardInterrupt: 